# In-depth Analysis of Successful Negotiation Cases

View complete details of successful negotiations, including:
- Complete conversation transcript
- Deal Terms reached
- Memory (M) state
- Plan (P) strategy
- Negotiation process analysis

## 1. Setup and Imports

In [81]:
import sqlite3
import pandas as pd
import json
from datetime import datetime
from IPython.display import display, HTML, Markdown
import warnings
warnings.filterwarnings('ignore')

print("✅ Import successful")

✅ Import successful


## 2. Connect to Database

In [219]:
# Modify the path here
DB_PATH = "negotiations (2).db"  # Change to your database path

conn = sqlite3.connect(DB_PATH)
print(f"✅ Connected to database: {DB_PATH}")

✅ Connected to database: negotiations (2).db


In [104]:
# Check what time columns exist
query = "PRAGMA table_info(negotiation_sessions)"
columns_info = pd.read_sql_query(query, conn)
print("negotiation_sessions table columns:")
display(columns_info[['name', 'type']])

negotiation_sessions table columns:


,name,type
0,session_id,TEXT
1,major,TEXT
2,gender,TEXT
3,negotiation_experience,TEXT
4,scenario_name,TEXT
5,student_role,TEXT
6,ai_role,TEXT
7,ai_model,TEXT
8,student_goes_first,BOOLEAN
9,use_memory,BOOLEAN


In [90]:
# Check time format
query = """
SELECT 
    session_id,
    created_at,
    updated_at,
    total_rounds,
    deal_reached
FROM negotiation_sessions
LIMIT 5
"""

sample_times = pd.read_sql_query(query, conn)
display(sample_times)

,session_id,created_at,updated_at,total_rounds,deal_reached
0,bbb6c9f7-95c9-4cd8-a1dc-5080ab5ce03e,2025-12-10T00:04:45.396370,2025-12-10T00:04:45.396541,6,0
1,20b2f876-42ae-4be1-a9b5-a9347f3b50af,2025-12-10T00:06:12.949653,2025-12-10T00:06:12.949842,6,0
2,67f91417-c064-44ff-b0e9-65404a64f8df,2025-12-10T00:05:23.120772,2025-12-10T00:06:38.184273,6,0
3,362cd512-ee83-4ed6-bf80-8d5ea8ce6f49,2025-12-10T00:09:44.182842,2025-12-10T00:16:05.886083,6,0
4,7271f344-12b8-46cf-8892-f69ebec7dde3,2025-12-10T00:11:36.215731,2025-12-10T00:16:14.141166,6,1


In [105]:
# Get all negotiation records with duration
query = """
SELECT 
    session_id,
    scenario_name,
    student_role,
    ai_model,
    use_memory,
    use_plan,
    deal_reached,
    total_rounds,
    created_at,
    updated_at
FROM negotiation_sessions
ORDER BY created_at DESC
"""

all_df = pd.read_sql_query(query, conn)

# Convert to datetime and calculate duration
all_df['created_at'] = pd.to_datetime(all_df['created_at'])
all_df['updated_at'] = pd.to_datetime(all_df['updated_at'])
all_df['duration_seconds'] = (all_df['updated_at'] - all_df['created_at']).dt.total_seconds()
all_df['duration_minutes'] = all_df['duration_seconds'] / 60

print(f"\n📊 Total: {len(all_df)} negotiation records")
print(f"✅ Successful: {len(all_df[all_df['deal_reached']==1])} records")
print(f"❌ Failed: {len(all_df[all_df['deal_reached']==0])} records\n")

# Display list
display_df = all_df.copy()
display_df['session_id_short'] = display_df['session_id'].str[:8]
display_df['M+P'] = display_df.apply(
    lambda x: '✅' if x['use_memory'] and x['use_plan'] else '❌', 
    axis=1
)
display_df['deal'] = display_df['deal_reached'].apply(
    lambda x: '✅' if x == 1 else '❌'
)
display_df['duration_min'] = display_df['duration_minutes'].round(1)

display(display_df[['session_id_short', 'scenario_name', 
                     'ai_model', 'M+P', 'deal', 'total_rounds', 
                     'duration_min', 'created_at']])

print("\n💡 Copy session_id_short to view details")
print(f"\n⏱️ Average duration: {all_df['duration_minutes'].mean():.1f} minutes")
print(f"⏱️ Median duration: {all_df['duration_minutes'].median():.1f} minutes")


📊 Total: 28 negotiation records
✅ Successful: 2 records
❌ Failed: 26 records



,session_id_short,scenario_name,ai_model,M+P,deal,total_rounds,duration_min,created_at
0,59abb8e6,Main_Street,openai/gpt-5,❌,❌,6,0.0,2025-12-10 04:22:05.373531
1,e5b6107b,Main_Street,openai/gpt-5,❌,❌,6,0.6,2025-12-10 04:22:02.791032
2,d1c19ae5,Main_Street,openai/gpt-5,✅,❌,6,1.0,2025-12-10 04:22:00.328075
3,ac9d1e05,Main_Street,openai/gpt-5,✅,❌,6,0.0,2025-12-10 04:21:58.003536
4,7b0da794,Main_Street,openai/gpt-5,❌,❌,6,0.0,2025-12-10 04:16:29.637320
5,865142bf,Main_Street,openai/gpt-5,✅,❌,6,0.0,2025-12-10 03:50:27.520379
6,c38e57af,Main_Street,openai/gpt-5,❌,❌,6,0.0,2025-12-10 03:50:25.332368
7,a9418128,Main_Street,openai/gpt-5,✅,❌,6,1.1,2025-12-10 03:50:23.652288
8,05013a9d,Main_Street,openai/gpt-5,❌,❌,6,0.6,2025-12-10 03:50:21.542086
9,c0f1968e,Main_Street,openai/gpt-5,❌,❌,6,4.8,2025-12-10 02:16:43.085990



💡 Copy session_id_short to view details

⏱️ Average duration: 5.2 minutes
⏱️ Median duration: 0.0 minutes


## 3. List All Successful Negotiations

In [75]:
# Get all successful negotiations
query = """
SELECT 
    session_id,
    student_name,
    scenario_name,
    student_role,
    ai_model,
    use_memory,
    use_plan,
    created_at,
    feedback_text TEXT
FROM negotiation_sessions
WHERE deal_reached = 1
ORDER BY created_at DESC
"""

successful_df = pd.read_sql_query(query, conn)

print(f"\n📊 Found {len(successful_df)} successful negotiations\n")

# Display list
display_df = successful_df.copy()
display_df['session_id_short'] = display_df['session_id'].str[:8]
display_df['M+P'] = display_df.apply(
    lambda x: '✅' if x['use_memory'] and x['use_plan'] else '❌', 
    axis=1
)

display(display_df[['session_id_short', 'student_name', 'scenario_name', 
                     'ai_model', 'M+P', 'created_at']])

print("\n💡 Copy session_id_short to view details")


📊 Found 11 successful negotiations



,session_id_short,student_name,scenario_name,ai_model,M+P,created_at
0,6bcbd0fe,Chris,Main_Street,openai/gpt-5,❌,2025-12-01T06:59:07.400156
1,03b5791c,Chris,Main_Street,openai/gpt-5,✅,2025-12-01T06:54:53.616963
2,06eb1797,Chris,Top_talent,openai/gpt-5,❌,2025-12-01T06:54:01.765574
3,d9e5ab85,Chris,Top_talent,openai/gpt-5,❌,2025-12-01T06:53:49.801746
4,2d6e0616,Chris,Main_Street,openai/gpt-5,❌,2025-12-01T06:17:38.355369
5,30c3c80c,Chris,Main_Street,openai/gpt-5,❌,2025-12-01T06:17:23.112858
6,746dbd3e,Chris,Top_talent,openai/gpt-5,✅,2025-11-30T23:21:57.375665
7,05a9b9d7,Chris,Top_talent,openai/gpt-5,✅,2025-11-30T23:21:51.933084
8,c5ab236a,Chris,Top_talent,openai/gpt-5,✅,2025-11-30T23:21:49.456787
9,21d35936,Chris,Top_talent,openai/gpt-5,✅,2025-11-30T23:21:41.849373



💡 Copy session_id_short to view details


In [71]:
query = """
SELECT 
    student_name,
    COUNT(*) as count
FROM negotiation_sessions
GROUP BY student_name
ORDER BY count DESC
"""

name_counts = pd.read_sql_query(query, conn)
print(f"📊 总共有 {len(name_counts)} 个不同的学生\n")
display(name_counts)

📊 总共有 8 个不同的学生



,student_name,count
0,Chris,72
1,Francisco Fernandez,3
2,Zihao Sun,2
3,Catherine Xu,2
4,AL,2
5,sasha,1
6,Runzhi Jin,1
7,Prof Cain,1


## 4. View Specific Case Details

### 👇 Enter the session_id you want to view below (first 8 characters are sufficient)

In [221]:
# ============================================
# 👉 MODIFY HERE: Enter the session_id you want to view
# ============================================
SESSION_ID = "2d84161a-af45-4a9f-8f5d-509b33c6c5fc"  # Change to the actual session_id (first 8 characters)

# If only first 8 characters, find the complete ID
if len(SESSION_ID) == 8:
    full_id_query = f"""
    SELECT session_id FROM negotiation_sessions 
    WHERE session_id LIKE '{SESSION_ID}%' AND deal_reached = 1
    """
    result = pd.read_sql_query(full_id_query, conn)
    if len(result) > 0:
        SESSION_ID = result['session_id'].iloc[0]
        print(f"✅ Found complete ID: {SESSION_ID}")
    else:
        print(f"❌ No successful cases found starting with {SESSION_ID} ")

# Get complete data
query = f"""
SELECT * FROM negotiation_sessions
WHERE session_id = '{SESSION_ID}'
"""

case_df = pd.read_sql_query(query, conn)

if len(case_df) == 0:
    print(f"❌ session_id not found: {SESSION_ID}")
else:
    case = case_df.iloc[0]
    print(f"\n✅ Case loaded: {SESSION_ID}")


✅ Case loaded: 2d84161a-af45-4a9f-8f5d-509b33c6c5fc


## 5. 📋 Case Overview

In [222]:
if len(case_df) > 0:
    print("=" * 80)
    print("                          Case Overview")
    print("=" * 80)
    
    print(f"\n🆔 Session ID:     {case['session_id']}")
    print(f"🎯 Scenario:           {case['scenario_name']}")
    print(f"🎭 Student Role:       {case['student_role']}")
    print(f"🤖 AI Role:         {case['ai_role']}")
    print(f"🧠 AI Model:         {case['ai_model']}")
    print(f"🔧 Memory Enabled:     {'Yes ✅' if case['use_memory'] else 'No ❌'}")
    print(f"📋 Plan Enabled:       {'Yes ✅' if case['use_plan'] else 'No ❌'}")
    print(f"🎲 Who Starts First:       {'student' if case['student_goes_first'] else 'AI'}")
    print(f"🔄 Total Rounds:       {case['total_rounds']}")
    
    # 计算时长
    try:
        start = pd.to_datetime(case['created_at'])
        end = pd.to_datetime(case['updated_at'])
        duration = (end - start).total_seconds() / 60
        print(f"⏳ Duration:           {duration:.1f} minutes")
    except:
        pass
    
    # 消息统计
    try:
        transcript = json.loads(case['transcript'])
        print(f"💬 Total Messages:       {len(transcript)}")
        print(f"📅 Start Time:       {case['created_at']}")
        print(f"✅ Completion Time:       {case['updated_at']}")
    except:
        pass

                          Case Overview

🆔 Session ID:     2d84161a-af45-4a9f-8f5d-509b33c6c5fc
🎯 Scenario:           Top_talent
🎭 Student Role:       side2
🤖 AI Role:         side1
🧠 AI Model:         openai/gpt-5
🔧 Memory Enabled:     Yes ✅
📋 Plan Enabled:       Yes ✅
🎲 Who Starts First:       student
🔄 Total Rounds:       6
⏳ Duration:           87.2 minutes
💬 Total Messages:       6
📅 Start Time:       2025-12-10T05:24:19.668020
✅ Completion Time:       2025-12-10T06:51:32.012236


## 6. 🤝 Deal Terms Reached

In [213]:
if len(case_df) > 0:
    print("=" * 80)
    print("                          Deal Terms")
    print("=" * 80)
    
    print("\n📝 Student's Proposed Deal Terms:")
    print("-" * 80)
    if case['student_deal_json']:
        try:
            student_deal = json.loads(case['student_deal_json'])
            print(json.dumps(student_deal, indent=2, ensure_ascii=False))
        except Exception as e:
            print(f"Parse error: {e}")
    else:
        print("(Not recorded)")
    
    print("\n\n🤖 AI Confirmed Deal Terms:")
    print("-" * 80)
    if case['ai_deal_json']:
        try:
            ai_deal = json.loads(case['ai_deal_json'])
            print(json.dumps(ai_deal, indent=2, ensure_ascii=False))
        except Exception as e:
            print(f"Parse error: {e}")
    else:
        print("(Not recorded)")
    
    # 对比差异
    if case['student_deal_json'] and case['ai_deal_json']:
        try:
            student_deal = json.loads(case['student_deal_json'])
            ai_deal = json.loads(case['ai_deal_json'])
            
            print("\n\n🔍 Comparison Analysis:")
            print("-" * 80)
            
            all_keys = set(student_deal.keys()) | set(ai_deal.keys())
            
            for key in sorted(all_keys):
                student_val = student_deal.get(key, "(Missing)")
                ai_val = ai_deal.get(key, "(Missing)")
                
                if student_val == ai_val:
                    print(f"✅ {key:30s}: {student_val}")
                else:
                    print(f"⚠️  {key:30s}:")
                    print(f"   Student: {student_val}")
                    print(f"   AI:   {ai_val}")
        except:
            pass

                          Deal Terms

📝 Student's Proposed Deal Terms:
--------------------------------------------------------------------------------
(Not recorded)


🤖 AI Confirmed Deal Terms:
--------------------------------------------------------------------------------
(Not recorded)


In [214]:
case["ai_memory_history"]

'[{"round": "1.2", "content": "OFFERS: [Us: Start Date: not yet; Work Location: not yet; Base Salary: not yet; Remote Work Policy: not yet; Signing Bonus: not yet; Vacation Time: not yet; Blair Rotation: not yet; Them: Start Date: not yet; Work Location: not yet; Base Salary: not yet; Remote Work Policy: not yet; Signing Bonus: not yet; Vacation Time: not yet; Blair Rotation: not yet; Them-best-for-us: none yet]\\nOPPONENT PATTERNS: [Opened by asking for our priorities/constraints; no offers or commitments; exploratory inquiry]\\nOPPONENT PRIORITIES: [Not yet Stated; Hypothesis: interested in 3-month Blair DC rotation]\\nOPPONENT CONSTRAINTS: [not yet]"}, {"round": "2.2", "content": "OFFERS: [Us: Start Date: not yet\\u2192July 5 [updated]; Work Location: not yet\\u2192New Haven, CT [updated]; Base Salary: not yet\\u2192$135,000 [updated]; Remote Work Policy: not yet\\u21920 day/week [updated]; Signing Bonus: not yet\\u2192$5,000 [updated]; Vacation Time: not yet\\u21922 weeks [updated]

In [216]:
case["ai_plan_history"]

'[{"round": "1.2", "content": "- ROUND GOAL: Strong-anchor a bundled offer; secure tentative agreement on Location (New Haven) and Start Date (July 5) while probing candidate\\u2019s top 2 priorities and stance on Blair rotation.\\n- KEY LEVERS: Push: New Haven; 2 weeks vacation; 0 remote; July 5 start. Tradeables: salary up toward $138k; signing bonus up to $15k; remote up to 1 day; start to July 15. Firm: No 3\\u2011month Blair rotation (offer non-contract mentorship alternatives).\\n- TACTICS: Open with brief rationale (team integration, CEO conference, equity across roles). Present full package. Ask: \\u201cWhat are your top two priorities?\\u201d and \\u201cIs the Blair DC rotation a must-have or nice-to-have if we ensure direct access to Blair?\\u201d Use if/then bundles (e.g., 1 remote day \\u2194 lower bonus; July 15 start \\u2194 salary stays at $132k). Seek provisional yes on Location and Start.\\n- OFFER SCAFFOLD: Start Date: July 5; Work Location: New Haven, CT; Base Salary

## 7. 🧠 Memory (M) State

In [223]:
if len(case_df) > 0:
    print("=" * 80)
    print("                     AI Memory (Final State)")
    print("=" * 80)
    
    if not case['use_memory']:
        print("\n❌ Memory not enabled for this case")
    elif not case['ai_memory']:
        print("\n(Memory state not recorded)")
    else:
        print("\n🧠 AI Memory recorded the following information:\n")
        print(case['ai_memory'])
        
        # 分析Memory内容
        memory_text = case['ai_memory']
        print("\n\n📊 Memory Analysis:")
        print("-" * 80)
        print(f"Character count: {len(memory_text)}")
        print(f"Line count: {len(memory_text.split('\n'))}")
        
        # 检查关键词
        keywords = ['OFFERS', 'PATTERNS', 'PRIORITIES', 'CONSTRAINTS']
        found = [kw for kw in keywords if kw in memory_text]
        print(f"Key sections included: {', '.join(found)}")

                     AI Memory (Final State)

🧠 AI Memory recorded the following information:

OFFERS: [Us: Start Date: July 5 [unchanged]; Work Location: New Haven, CT [unchanged]; Base Salary: $135,000 [unchanged]; Remote Work Policy: 0 day/week [unchanged]; Signing Bonus: $5,000 [unchanged]; Vacation Time: 2 weeks [unchanged]; Blair Rotation: No rotation [unchanged]; Alt: Jul 15 for $0 sign-on (base $135k) OR 1 day/week remote for $0 sign-on (base $135k); choose one, not both; Them: Start Date: not yet; Work Location: not yet; Base Salary: not yet; Remote Work Policy: not yet; Signing Bonus: not yet; Vacation Time: not yet; Blair Rotation: not yet; Them-best-for-us: none]

OPPONENT PATTERNS: [Repeatedly asks for our internal priorities/scoring; declines to state their own priorities; no counteroffers; no tentative locks; info-seeking and transparency requests; no acceptance of any issue]

OPPONENT PRIORITIES: [Stated: not yet; Hypothesis: Interest in 3-month Blair DC rotation remain

## 8. 📋 Plan (P) Strategy

In [224]:
if len(case_df) > 0:
    print("=" * 80)
    print("                   AI Strategic Plan (Final State)")
    print("=" * 80)
    
    if not case['use_plan']:
        print("\n❌ Plan not enabled for this case")
    elif not case['ai_plan']:
        print("\n(Plan state not recorded)")
    else:
        print("\n📋 AI's Final Strategic Plan:\n")
        print(case['ai_plan'])
        
        # 分析Plan内容
        plan_text = case['ai_plan']
        print("\n\n📊 Plan Analysis:")
        print("-" * 80)
        print(f"Character count: {len(plan_text)}")
        print(f"Line count: {len(plan_text.split('\n'))}")
        
        # 检查关键词
        keywords = ['ROUND GOAL', 'KEY LEVERS', 'TACTICS', 'OFFER SCAFFOLD', 'RISK']
        found = [kw for kw in keywords if kw in plan_text]
        print(f"Strategy sections included: {', '.join(found)}")

                   AI Strategic Plan (Final State)

📋 AI's Final Strategic Plan:

- ROUND GOAL: Lock NH, July 5 start, 0 remote, 2 weeks PTO, and No Blair rotation; force a choice between two comp packages and secure a tentative yes.
- KEY LEVERS: Push: Location (NH), Start (Jul 5), Remote (0), PTO (2w), No Blair. Trade: Base ($132k–$135k) vs Signing ($0–$10k). Conditional: 1 remote day OR Jul 15 (not both) tied to $0 sign-on.
- TACTICS: Shift from info-sharing to decision; present two balanced packages; frame fairness/mentorship/CEO event; deflect Blair rotation via scheduled mentorship; ask a closing question; seek tentative locks on structure today.
- TACTICS: If they won’t choose, require they name their top two must-haves (base vs bonus vs remote/start), then lock structural issues on-record.
- OFFER SCAFFOLD: Option A — New Haven; July 5; Base $133,000; Signing Bonus $3,000; 0 remote; 2 weeks vacation; No Blair rotation.
- OFFER SCAFFOLD: Option B — New Haven; July 5; Base $135,0

## 9. 💬 Complete Conversation Transcript（Transcript）

In [191]:
if len(case_df) > 0:
    print("=" * 80)
    print("                         Complete Conversation Transcript")
    print("=" * 80)
    
    try:
        transcript = json.loads(case['transcript'])
        total_messages = len(transcript)
        
        print(f"\n💬 共 {total_messages} 条消息\n")
        print("=" * 80)
        
        for i, message in enumerate(transcript, 1):
            # 解析消息格式: "Round X.Y - Label: content"
            if " - " in message:
                parts = message.split(" - ", 1)
                round_info = parts[0]
                
                if len(parts) > 1 and ":" in parts[1]:
                    speaker_content = parts[1].split(":", 1)
                    speaker = speaker_content[0].strip()
                    content = speaker_content[1].strip() if len(speaker_content) > 1 else ""
                    
                    # 判断是学生还是AI
                    if case['student_role'] in speaker or "Student" in speaker:
                        icon = "👤"
                        print(f"\n{icon} [{round_info}] {speaker}:")
                    else:
                        icon = "🤖"
                        print(f"\n{icon} [{round_info}] {speaker}:")
                    
                    # 打印内容（处理长消息）
                    if len(content) > 500:
                        print(f"   {content[:2000]}...")
                        print(f"   ... (共 {len(content)} characters)")
                    else:
                        print(f"   {content}")
                else:
                    print(f"\n💬 [{round_info}]")
                    print(f"   {parts[1] if len(parts) > 1 else message}")
            else:
                print(f"\n💬 消息 {i}:")
                if len(message) > 500:
                    print(f"   {message[:500]}...")
                    print(f"   ... (共 {len(message)} characters)")
                else:
                    print(f"   {message}")
            
            print("-" * 80)
        
    except Exception as e:
        print(f"\n❌ 解析transcript出错: {e}")

                         Complete Conversation Transcript

💬 共 2 条消息


🤖 [Round 1.1] The Candidate:
   what's your priorities and constraints?
--------------------------------------------------------------------------------

🤖 [Round 1.2] The Recruiter:
   I’m thrilled about you joining us—your NYC marketing experience plus the biochem background is exactly what we need. Let me lay out our priorities so we can focus the discussion and move quickly.

- Timing: Ideal start is July 5 so you’re present for the CEO’s internal online conference and the H2 digital launch cadence. July 15 could be workable with the right plan; later starts are much harder.
- Location: New Haven, CT is our top priority and where the core digital team sits. Seoul is a distant second; Philadelphia would be difficult to approve given current staffing.
- In‑office presence: We’re a mentorship-heavy culture. Baseline is on-site (no default remote days), especially in the first months. We can revisit limited flexibil

In [106]:
case['feedback_text']

'═══════════════════════════════════════\n📊 PERFORMANCE SUMMARY\n═══════════════════════════════════════\n\n- Final outcome: 31,000 points achieved\n- Overall assessment: You secured a deal above your BATNA of 0 points, which is a positive result under time constraints. However, you left significant value on the table by conceding on high-point issues early and not pushing harder on key financial terms.\n\n═══════════════════════════════════════\n🎯 STRATEGIC ANALYSIS\n═══════════════════════════════════════\n\n- **Start Date (15,000 points achieved out of 15,000 possible)**: You secured the maximum value by holding firm on August 1st or later. Your consistent emphasis on personal timing needs paid off as the recruiter conceded early. This was a strong tactical move, ensuring a high-value win without trade-offs.\n  \n- **Work Location (0 points achieved out of 4,000 possible)**: You conceded to New Haven (0 points) despite Philadelphia offering 4,000 points. While you articulated person

## 10. 📊 Negotiation Process Analysis

In [58]:
if len(case_df) > 0:
    print("=" * 80)
    print("                        Negotiation Process Analysis")
    print("=" * 80)
    
    try:
        transcript = json.loads(case['transcript'])
        total_messages = len(transcript)
        
        # 统计消息长度
        message_lengths = []
        student_messages = 0
        ai_messages = 0
        deal_signals = 0
        
        for msg in transcript:
            # 计算消息长度
            if " - " in msg:
                parts = msg.split(" - ", 1)
                if len(parts) > 1:
                    msg_content = parts[1]
                    message_lengths.append(len(msg_content))
            else:
                message_lengths.append(len(msg))
            
            # 统计发言者
            if case['student_role'] in msg:
                student_messages += 1
            elif case['ai_role'] in msg:
                ai_messages += 1
            
            # 检查deal信号
            if "$DEAL_REACHED$" in msg:
                deal_signals += 1
        
        print("\n📊 Basic Statistics:")
        print("-" * 80)
        print(f"Total Messages:        {total_messages}")
        print(f"Student Messages:        {student_messages}")
        print(f"AI Messages:          {ai_messages}")
        print(f"完成Round数:      {total_messages // 2}")
        print(f"$DEAL_REACHED$:  出现 {deal_signals} 次")
        
        if message_lengths:
            print("\n📏 Message Complexity:")
            print("-" * 80)
            print(f"Average Message Length:    {sum(message_lengths)/len(message_lengths):.0f} characters")
            print(f"Shortest Message:        {min(message_lengths)} characters")
            print(f"Longest Message:        {max(message_lengths)} characters")
        
        # 计算每Round平均时长
        try:
            start = pd.to_datetime(case['created_at'])
            end = pd.to_datetime(case['updated_at'])
            total_minutes = (end - start).total_seconds() / 60
            rounds = total_messages // 2
            
            if rounds > 0:
                print("\n⏱️  Time Analysis:")
                print("-" * 80)
                print(f"Total Duration:          {total_minutes:.1f} minutes")
                print(f"每Round平均:      {total_minutes/rounds:.1f} minutes")
                print(f"Average Per Message:    {total_minutes/total_messages:.1f} minutes")
        except:
            pass
        
        # 判断谈判风格
        print("\n🎯 Negotiation Characteristics:")
        print("-" * 80)
        
        avg_length = sum(message_lengths)/len(message_lengths) if message_lengths else 0
        if avg_length > 500:
            print("💬 Message Style:      Detailed (longer messages)")
        elif avg_length > 200:
            print("💬 Message Style:      Moderate")
        else:
            print("💬 Message Style:      Concise (shorter messages)")
        
        if total_minutes / rounds < 3 if rounds > 0 else False:
            print("⚡ Negotiation Pace:      快速（<3minutes/Round）")
        elif total_minutes / rounds < 5 if rounds > 0 else False:
            print("⚡ Negotiation Pace:      Normal（3-5minutes/Round）")
        else:
            print("⚡ Negotiation Pace:      缓慢（>5minutes/Round）")
        
        if rounds < case['total_rounds'] / 2:
            print("🎉 Deal Efficiency:      高（不到一半Round就达成）")
        elif rounds < case['total_rounds']:
            print("🎉 Deal Efficiency:      Normal")
        else:
            print("🎉 Deal Efficiency:      压线（用完所有Round）")
        
    except Exception as e:
        print(f"\n❌ Analysis error: {e}")

## 11. 🔍 Quick View: Comparison of Recent 3 Successful Cases

In [19]:
# 获取最近3个成功案例
recent_query = """
SELECT 
    session_id,
    student_name,
    scenario_name,
    ai_model,
    use_memory,
    use_plan,
    transcript,
    created_at,
    updated_at
FROM negotiation_sessions
WHERE deal_reached = 1
ORDER BY created_at DESC
LIMIT 3
"""

recent_df = pd.read_sql_query(recent_query, conn)

if len(recent_df) > 0:
    print("=" * 80)
    print("                 Comparison of Recent 3 Successful Cases")
    print("=" * 80)
    
    comparison_data = []
    
    for idx, row in recent_df.iterrows():
        try:
            transcript = json.loads(row['transcript'])
            msg_count = len(transcript)
            
            start = pd.to_datetime(row['created_at'])
            end = pd.to_datetime(row['updated_at'])
            duration = (end - start).total_seconds() / 60
            
            comparison_data.append({
                'Session': row['session_id'][:8],
                'Student': row['student_name'],
                'Scenario': row['scenario_name'],
                'Model': row['ai_model'].split('/')[-1],
                'M+P': '✅' if row['use_memory'] and row['use_plan'] else '❌',
                'Messages': msg_count,
                'Duration(min)': f"{duration:.1f}",
                'Rounds': msg_count // 2
            })
        except:
            pass
    
    if comparison_data:
        comparison_df = pd.DataFrame(comparison_data)
        display(comparison_df)
else:
    print("\n❌ No successful cases found")

                 Comparison of Recent 3 Successful Cases


,Session,Student,Scenario,Model,M+P,Messages,Duration(min),Rounds
0,6bcbd0fe,Chris,Main_Street,gpt-5,❌,10,26.1,5
1,03b5791c,Chris,Main_Street,gpt-5,✅,13,36.3,6
2,06eb1797,Chris,Top_talent,gpt-5,❌,8,23.0,4


## 12. Close Database Connection

In [13]:
conn.close()
print("✅ Database connection closed")

✅ Database connection closed
